In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Plots
using ProgressMeter
import ForwardDiff as FD
import Convex as cvx
import ECOS

include(joinpath(@__DIR__,"utils","fmincon.jl"))

  Activating environment at `c:\CMU_Spring\OptimalSailboat\Project.toml`


fmincon (generic function with 1 method)

In [1]:
"""
Converts from cartesian (x,y) to polar (r,θ) coordinates.
"""
function car2pol(x, y)
    r = √(x^2 + y^2)
    θ = atan(y, x)
    return r, θ
end;

"""
Wraps an angle (in radians) to the range [-π, π].
"""
function wrap2pi(θ)
    return θ - 2*π*floor((θ+π)/(2*π))
end;

"""
Converts from a vector of vectors to a matrix.
"""
function vec2mat(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end;

"""
Converts from a matrix to a vector of vectors.
"""
function mat2vec(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end;

### Variable Definitions

### State Variables:

- x: Position in the longitudinal (forward-backward) direction.
- y: Position in the lateral (side-to-side) direction.
- $\psi$: Roll angle
- $\phi$: Yaw angle

- u: Linear velocity in the longitudinal (forward-backward) direction.
- v: Linear Velocity in the lateral (side-to-side) direction.
- p: angular velocity roll
- r: angular velocity yaw


#### Control Variables:
- $\lambda$ - sail angle
 - $\sigma$ - rudder angle
 

#### Wind and Force Variables:
 - $a_{tw}$ - true wind speed
 - $\psi_{tw}$ - true wind direction
 - $g_s$ - force from sail
 - $g_r$ - force from rudder

#### Parameters:
 - $p_1$ - drift coefficient
 - $p_2$ - tangential friction
 - $p_3$ - angular friction
 - $p_4$ - sail lift
 - $p_5$ - rudder lift
 - $p_6$ - distance to sail CoE (center of effort)
 - $p_7$ - distance to mast
 - $p_8$ - distance to rudder
 - $p_9$ - mass of boat
 - $p_{10}$ - moment of inertia
 - $p_{11}$ - rudder break coefficient
 

In [ ]:
function J(ϕ,ψ)
    J = [cos(ψ) -sin(ψ)*cos(ϕ)  0 0;
         sin(ψ) cos(ψ)*cos(ϕ)  0 0;
         0 0 1 0;
         0 0 0 cos(ϕ)]
    return J
end;

function J2D(ϕ,ψ)
    J = [cos(ψ) -sin(ψ)*cos(ϕ);
         sin(ψ) cos(ψ)*cos(ϕ)]
    return J
end;

function MRB(m, Ixx, Izz)
    M = [ m 0 0 0;
          0 m 0 0;
          0 0 Ixx 0;
          0 0 0 Izz]
    return M
end;

function MA(X_u̇, Y_v̇, K_ṗ, N_ṙ)
    MA = -diagonal([X_u̇, Y_v̇, K_ṗ, N_ṙ])
    return MA
end;

function CRB(v)
    m = v[1]
    r = v[2]
    
    C = [0 -m*r 0 0;
         m*r 0 0 0;
         0 0 0 0;
         0 0 0 0]
    return C
end;

function CA(v, Y_v̇, X_u̇) 

    C = [ 0 0 0 Y_v̇*v;
        0 0 0 -X_u̇*v;
        0 0 0 0;
        -Y_v̇*v X_u̇*v 0 0]
    return C
end;

function Dl(d11, d22, d66)
    Dl = [d11 0 0 0;
          0 d22 0 0;
          0 0 0 0;
          0 0 0 d66]
    return Dl
end;

function Dq()
    
end


Hydrodynamic force is a result of water pressing on the hull. Total resistance contains three components. First one is frictional drag. It is described by the following formula:

$$ F_{hf} = \frac{1}{2} \rho v^2 SC_f $$

Where rho is the density of water, v is the speed of the boat, S is the surface area of the submerged part of the hull and Cf is the coefficient of frictional resistance.

Another two components of total resistance are pressure drag and the wave drag. The first one is caused by relatively lowpressure area behind the boat. The wave drag exists because the yacht transfers a part of its kinetic energy on the water and creates waves. Both of them are very
difficult to calculate and they are inter-dependent. They are often considered to be just one value: 

$$ F_{hr} = F_{hp} + F_{hw} = \frac{1}{2} \rho v^2 SC_{r} $$

Where Cr is the coefficient of pressure and wave resistance 

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4581575